In [12]:
import pandas as pd
from sklearn import preprocessing, decomposition, tree, model_selection, ensemble
import numpy as np

In [13]:
train_data = pd.read_csv('/Users/bharath/Desktop/Folder/Data Science/Projects/prudential-life-insurance-assessment/train.csv')
test_data = pd.read_csv('/Users/bharath/Desktop/Folder/Data Science/Projects/prudential-life-insurance-assessment/test.csv')

total_data = pd.concat([train_data, test_data])
train_data.shape
#print(total_data)
train_data.shape

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


(59381, 128)

In [17]:
total_data.shape
train_data.info()
total_data1 = total_data.drop(['Product_Info_2','Medical_History_1', 'Medical_History_10', 'Medical_History_15', 'Medical_History_24', 'Medical_History_32'], axis=1)
total_data1.shape
print(total_data['Product_Info_2'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
D3    18753
D4    14071
A8     9140
D1     8611
D2     8344
E1     3711
A1     3219
A2     3072
A6     2733
A7     1823
A3     1564
B2     1446
A5     1009
C3      437
C1      377
C4      291
A4      263
C2      197
B1       85
Name: Product_Info_2, dtype: int64


In [4]:
# Check for numeric and chategorical columns
#previous_num_columns = total_data1.select_dtypes(exclude=['object']).columns.values.tolist()
#print(previous_num_columns)

In [18]:
Imputer = preprocessing.Imputer(strategy='mean')
Imputer.fit(total_data1)
X = Imputer.transform(total_data1)
total_data_tr = pd.DataFrame(X, columns=total_data1.columns)
#total_missing = total_data_tr.isnull().any().sum()
#print(total_missing)
#X.shape
y_impute = total_data_tr['Response']
x_impute = total_data_tr.drop('Response',axis=1)
y_train = y_impute[0:train_data.shape[0]]

/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
Scaler = preprocessing.StandardScaler()
Scaler.fit(x_impute)
X_scaled = Scaler.transform(x_impute)
X_scaled_df = pd.DataFrame(X_scaled, columns = x_impute.columns)
X_scaled.shape
total_missing1 = X_scaled_df.isnull().any().sum()
print(total_missing1)

0


In [20]:
pca = decomposition.PCA(n_components=10)
pca.fit(X_scaled)
x_transformed = pca.transform(X_scaled)
x_transformed.shape
#X_transformed = X_scaled

(79146, 10)

In [21]:
X_train = x_transformed[0:train_data.shape[0]]
X_train.shape
X_test = x_transformed[train_data.shape[0]:]
X_test.shape
X_train.shape
#total_data_X = pd.DataFrame(X_train, columns=total_data1.columns)
#total_missing = total_data_tr.isnull().any().sum()
#print(total_missing)

(59381, 10)

In [23]:
#dt = tree.DecisionTreeClassifier()
#dt_ada = ensemble.AdaBoostClassifier(dt,random_state=10)
#ada_grid = {'n_estimators':[5, 8, 10, 12],'learning_rate':[0.1, 0.5, 0.9]}
#param_grid = model_selection.GridSearchCV(dt_ada, ada_grid, cv=10, n_jobs=5)

In [26]:
#dt = tree.DecisionTreeClassifier()
#-------
#dt = ensemble.RandomForestClassifier(n_estimators=5)
#-------
dt = ensemble.RandomForestClassifier(random_state=5)
dt_rand_grid = {'n_estimators':[40,50],'criterion':['gini','entropy'],'max_depth': [2,4,6],'min_samples_split':[2,5,8] }
param_grid = model_selection.GridSearchCV(dt, dt_rand_grid, cv=5, n_jobs=5)
#----------
#param_grid = model_selection.GridSearchCV(dt, dt_rand_grid, cv=5, n_jobs=5)

In [27]:
param_grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=5, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'n_estimators': [40, 50], 'criterion': ['gini', 'entropy'], 'max_depth': [2, 4, 6], 'min_samples_split': [2, 5, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [25]:
param_grid.best_score_

0.3187383169700746

In [13]:
final_model = param_grid.best_estimator_
final_model.score(X_train, y_train)

0.42042067327933175

In [14]:
param_grid.score(X_train,y_train)

0.42042067327933175

In [54]:
cv_score = model_selection.cross_val_score(final_model, X_train, y_train, cv=15)
print(cv_score.mean())

0.40988065587617467


In [155]:
test_data['Response'] = dt.predict(X_test)

In [ ]:
#test_data.to_csv("Prudential.csv",columns=['Id','Survived'],index=False)